In [ ]:
import requests
import json
import mysql.connector as my_con
import datetime

In [ ]:
class Tabela:
    def _init_(self):
        self.NomTab = ""
        self.CamVal = {}
        self.ValFil = {}

def abrir_conexao():
    con = my_con.connect(host='localhost', database='osb_sp', user='root',password='xxxxxxxx') 
    if con.is_connected():
        db_info = con.get_server_info()
        print("Conectado ao servidor MySQL ", db_info)
        return con, con.cursor()

def fechar_conexao():
    if con.is_connected():
        cursor.close()
        con.close()
        print("Conexão com MySQL foi encerrada")
        
def insert(tabela):
    campos = tabela.CamVal.keys()
    valores = tabela.CamVal.values()
    url = 'INSERT INTO '+tabela.NomTab +'('+','.join(campos)+') VALUES('+','.join(valores)+');'
    print(url)
    cursor.execute(url)
    con.commit()

def update(tabela):
    str_set = ''
    for campo, valor in tabela.CamVal.items():
        str_set = str_set + campo +'='+valor+', '
    str_set = str_set[:-2]

    str_where = ''
    for campo, valor in tabela.ValFil.items():
        str_where = str_where + campo +'='+valor+' AND '
    str_where = str_where[:-5]

    url = 'UPDATE '+tabela.NomTab +' SET '+str_set+' WHERE '+str_where 
    print(url)
    cursor.execute(url)
    con.commit()

def delete(tabela):
    str_where = ''
    for campo, valor in tabela.ValFil.items():
        str_where = str_where + campo +'='+valor+' AND '
    str_where = str_where[:-5]

    url = 'DELETE FORM '+tabela.NomTab+' WHERE '+str_where 
    print(url)
    cursor.execute(url)
    con.commit()

def select(tabela):
    str_where = ''
    for campo, valor in tabela.ValFil.items():
        str_where = str_where + campo +'='+valor+' AND '
    str_where = str_where[:-5]

    url = 'SELECT * FROM '+tabela.NomTab
    if str_where != '':
        url =  url +' WHERE '+str_where 
    print(url)
    cursor.execute(url)
    records = cursor.fetchall()
    return records

In [ ]:
vereadores_raw = requests.get(
    'https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx/VereadoresCMSPJSON').json()

In [ ]:
con, cursor = abrir_conexao()

tabela  = Tabela()
tabela.NomTab = 'R001VER'
for ver_i in vereadores_raw:
    CodVer = ver_i['chave']
    NomVer = ver_i['nome']
    query_par = {'CodVer': str(CodVer), 'NomVer': "'"+NomVer+"'"}
    tabela.CamVal = query_par
    try:
        insert(tabela)
    except Exception as err:
        print(err)
    
fechar_conexao()

In [ ]:
con, cursor = abrir_conexao()

tabela  = Tabela()
tabela.NomTab = 'R001PAR'
partidos_aux = []
for i in vereadores_raw:
    for j in i['filiacoes']:
        if j['partido']['chave'] not in partidos_aux:
            partidos_aux.append(j['partido']['chave'])
            CodPar = j['partido']['chave']
            SigPar = j['partido']['sigla']
            DesPar = j['partido']['nome']
            query_par = {'CodPar': str(CodPar), 'SigPar': "'"+SigPar+"'", 'DesPar': "'"+DesPar+"'"}
            tabela.CamVal = query_par
            try:
                insert(tabela)
            except Exception as err:
                print(err)
    
fechar_conexao()

In [ ]:
con, cursor = abrir_conexao()

tabela  = Tabela()
tabela.NomTab = 'R001VPA'
for i in vereadores_raw:
    for j in i['mandatos']:
        CodVer = i['chave']
        CodPar = j['partido']['chave']
        DatIni = datetime.datetime.fromtimestamp(int(j['inicio'][6:-2])/1000.0, tz=datetime.timezone.utc).strftime('%Y/%m/%d')
        try:
            DatFim = datetime.datetime.fromtimestamp(int(j['fim'][6:-2]) /1000.0, tz=datetime.timezone.utc).strftime('%Y/%m/%d')
        except:
            DatFim = datetime.datetime.fromtimestamp(0).strftime('%Y/%m/%d')

        query_par = {'CodVer': str(CodVer), 'CodPar': str(CodPar), 'DatIni': "'"+DatIni+"'", 'DatFim': "'"+DatFim+"'"}
        tabela.CamVal = query_par
        try:
            insert(tabela)
        except Exception as err:
            print(err)
    
fechar_conexao()

In [ ]:
url = 'https://splegisws.saopaulo.sp.leg.br/'
anos = list(range(1991,2024))

In [ ]:
con, cursor = abrir_conexao()

tabela_1  = Tabela()
tabela_1.NomTab = 'R001TPR'

tabela_2  = Tabela()
tabela_2.NomTab = 'R001PRO'

tipos_aux = []

for ano in anos:
    projetos = []
    print('Obtendo projetos do ano de: '+ str(ano)+' ...')
    projetos_por_ano = '/ws/ws2.asmx/ProjetosPorAnoJSON?Ano='+str(ano)
    r = requests.get(url+projetos_por_ano)
    projetos_r = r.json()
    projetos_r

    for projeto_i in projetos_r:

        if projeto_i['_tipo']  not in tipos_aux:
            tipos_aux.append(projeto_i['_tipo'])
            CodTip = projeto_i['_tipo'] 
            DesTip = projeto_i['tipo'] 
            
            query_par_1 = {'CodTip': str(CodTip), 'DesTip': "'"+DesTip+"'"}
            tabela_1.CamVal = query_par_1
            try:
                insert(tabela_1)
            except Exception as err:
                print(err)
        
        CodPro = projeto_i['chave']
        CodTip = projeto_i['_tipo']
        NumPro = projeto_i['numero']
        DatPro = projeto_i['data']
        try:
            DesEme = projeto_i['ementa']
        except:
            DesEme = ''
        
        query_par_2 = {'CodPro': str(CodPro), 'CodTip': str(CodTip), 'NumPro': str(NumPro), 'DatPro': "'"+DatPro+"'"}
        tabela_2.CamVal = query_par_2
        try:
            insert(tabela_2)
        except Exception as err:
            print(err)
        
fechar_conexao()

In [ ]:
url = 'https://splegisws.saopaulo.sp.leg.br/'
projetos_autores = '/ws/ws2.asmx/ProjetosAutoresJSON'

r001tpr = Tabela()
r001tpr.NomTab = 'R001TPR'

r001pro = Tabela()
r001pro.NomTab = 'R001PRO'
r001pro.ValFil = {}

con, cursor = abrir_conexao()
try:
    projetos = select(r001pro)
except Exception as err:
    print(err)

r001vpr = Tabela()
r001vpr.NomTab = 'R001VPR'

for projeto in projetos:
    # Executar uma query na tabela r001tpr
    r001tpr.ValFil = {'CodTip': str(projeto[1])}

    try:
        tipo = select(r001tpr)
    except Exception as err:
        print(err)
   
    DesTip = tipo[0][1]

    AnoPro = projeto[3].year
    NumPro = projeto[2]
    
    CodPro = projeto[0]
    print(CodPro)
    if CodPro > 204549:
        query = (
            f'{projetos_autores}?ano={AnoPro}&tipo={DesTip}&numero={NumPro}')
        projeto_r = requests.get(url+query)
        projeto_j = projeto_r.json()

        try:
            for autor in projeto_j[0]['autores']:
                CodVer = autor['chave']

                query_par = {'CodVer': str(CodVer), 'CodPro': str(CodPro)}
                r001vpr.CamVal = query_par
                try:
                    insert(r001vpr)
                except Exception as err:
                    print(err)
        except Exception as err:
            print(err)
  
fechar_conexao()